# MESH Plot NO3 uptake

### Directories

In [1]:
GEM_folder = "..\\..\\..\\.."

In [2]:
modelfile = "model\\iMgadit23.json"

### Imports

In [3]:
# Imports
import cobra
from cobra.flux_analysis.parsimonious import pfba
import json
import numpy as np
import pandas as pd

from os.path import join as __join

import matplotlib.pyplot as plt


### Model

In [4]:
iMgadit23 = cobra.io.load_json_model(__join(GEM_folder, modelfile))

### Media

In [5]:
iMgadit23.reactions.NGAM_c.lower_bound

2.2

In [6]:
iMgadit23.reactions.EX_no3_.lower_bound = -1.25E-01

In [7]:
iMgadit23.reactions.EX_photon_abs_.lower_bound = -100
iMgadit23.objective = "BIOMASS_biomass_WT526_c"
solution = pfba(iMgadit23)
iMgadit23.summary()

C:\Local\Programmes\Anaconda3\envs\iMgadit884_python36\lib\site-packages\cobra\core\metabolite.py:105: UserWarning: 42.44882023256878 is not an integer (in formula C42.44882023256878H76.4292448299964O20.879111330619708N4.814750300170175P0.20036104118307208S0.14620606060000002Mg0.030432563599999997)
  % (count, self.formula)
C:\Local\Programmes\Anaconda3\envs\iMgadit884_python36\lib\site-packages\cobra\core\metabolite.py:105: UserWarning: 76.4292448299964 is not an integer (in formula C42.44882023256878H76.4292448299964O20.879111330619708N4.814750300170175P0.20036104118307208S0.14620606060000002Mg0.030432563599999997)
  % (count, self.formula)
C:\Local\Programmes\Anaconda3\envs\iMgadit884_python36\lib\site-packages\cobra\core\metabolite.py:105: UserWarning: 20.879111330619708 is not an integer (in formula C42.44882023256878H76.4292448299964O20.879111330619708N4.814750300170175P0.20036104118307208S0.14620606060000002Mg0.030432563599999997)
  % (count, self.formula)
C:\Local\Programmes\An

Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_,1.074,1,99.57%
fuc__L_e,EX_fuc__L_,0.0004813,6,0.27%
h2o_e,EX_h2o_,0.8986,0,0.00%
h_e,EX_h_,0.1278,0,0.00%
man_e,EX_man_,0.0002924,6,0.16%
mg2_e,EX_mg2_,0.000768,0,0.00%
no3_e,EX_no3_,0.125,0,0.00%
photon_abs_e,EX_photon_abs_,21.89,0,0.00%
pi_e,EX_pi_,0.005056,0,0.00%
so4_e,EX_so4_,0.004388,0,0.00%


# Phenotypic phase plan

### Function

In [8]:
def phenotypic_phase_plan(model,var1,var2,obj_rxn,file_name,min_axe,max_axe):
 
    # 1) Generate variables values
    res_dict = {}
    idx = 0
    matrice=pd.DataFrame(np.zeros((len(var1["values"]),len(var2["values"]))),index=var1["values"],columns=var2["values"]) 

    # 2) Set bounds
    for v1 in var1["values"]:
        # v1 flux
        model.reactions.get_by_id(var1["rxn_id"]).upper_bound = 0
        model.reactions.get_by_id(var1["rxn_id"]).lower_bound = -v1

        # v2 flux
        for v2 in var2["values"]:

            flag = True 

            model.reactions.get_by_id(var2["rxn_id"]).upper_bound = 0
            model.reactions.get_by_id(var2["rxn_id"]).lower_bound = -v2

        # 3) Simulation (pFBA)
            model.objective= obj_rxn

            try:
                sol = pfba(model)
                res_dict[idx] = [v1,v2,sol["EX_no3_"],sol["EX_co2_"],sol["EX_photon_abs_"],sol[obj_rxn]]
                matrice.at[v1,v2]= sol[obj_rxn]

            except :
                sol = -1
                res_dict[idx] = [sol,sol,sol,sol,sol,sol]
                matrice.at[v1,v2]= sol
            idx += 1

    # 4) save dataframes to csv
    res_df = pd.DataFrame.from_dict(res_dict,orient='index',columns=["Variable 1: "+ var1["rxn_id"],"Variable 1: " + var2["rxn_id"],"EX_no3_","EX_co2_","EX_photon_abs",obj_rxn + " flux"])
    res_df.to_csv(file_name+"DF.csv",sep=";")
    matrice.to_csv(file_name+"matrice.csv",sep=";")
    
    
    return(res_df,matrice)

## 1) NO3 vs photons

In [9]:
objective = "BIOMASS_biomass_WT526_c"

In [10]:
file = "PPP_no3_vs_photon" # to save all data


# NO3
variable_1 = {"metabo_id": "NO3",
             "rxn_id": "EX_no3_",
             "min": 0,
             "max":0.125,
             "step":0.005} 

variable_1["values"] = np.arange(variable_1["min"],variable_1["max"]+variable_1["step"],variable_1["step"])

# photons
variable_2 = {"metabo_id": "Photon uptake",
             "rxn_id": "EX_photon_abs_",
             "min": 0,
             "max":25,
             "step":1 }

variable_2["values"] = np.arange(variable_2["min"],variable_2["max"]+variable_2["step"],variable_2["step"])

    
file_name = file + "_" + objective[:-1]

# Reload model
iMgadit23 = cobra.io.load_json_model(__join(GEM_folder, modelfile))
# NO3 and CO2 unlimeted by default

res1 = phenotypic_phase_plan(iMgadit23,variable_1,variable_2,objective,file_name,min_axe=0,max_axe=25)

# 2) NO3 vs CO2

In [11]:
file = "PPP_no3_vs_co2" # to save all data

# NO3
variable_1 = {"metabo_id": "NO3",
             "rxn_id": "EX_no3_",
             "min": 0,
             "max":0.125,
             "step":0.00625}

variable_1["values"] = np.arange(variable_1["min"],variable_1["max"]+variable_1["step"],variable_1["step"])

# CO2
variable_2 = {"metabo_id": "CO2 uptake",
             "rxn_id": "EX_co2_",
             "min": 0,
             "max":1.2,
             "step":0.06 }

variable_2["values"] = np.arange(variable_2["min"],variable_2["max"]+variable_2["step"],variable_2["step"])

file_name = file + "_" + objective[:-1]

# Reload model
iMgadit23 = cobra.io.load_json_model(__join(GEM_folder, modelfile))
iMgadit23.reactions.EX_photon_abs_.lower_bound = -100000
res2 = phenotypic_phase_plan(iMgadit23,variable_1,variable_2,objective,file_name,min_axe=0,max_axe=25)